In [382]:
import numpy as np
from numpy import random as rnd
from matplotlib import pyplot as plt
import os,datetime,warnings,time

import tensorflow as tf
import pandas as pd
import seaborn as sns

from keras.models import Model,Sequential
from keras.layers import Dense,LSTM,Conv2D,Conv2DTranspose,Dropout,GRU,Activation,Add,Concatenate,BatchNormalization,Bidirectional,Embedding,Flatten

from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

import nltk
import string

import re

warnings.filterwarnings('ignore')
sns.set()

In [209]:
def split_sentence(basephrase,L,F):
    b = basephrase.split(' ')
    X = []
    Y = []
    for k in range(len(b)-L-F+1):
        X.append(b[k:k+L])
        Y.append(b[k+L:k+L+F])
    return X,Y

In [211]:
def build_labeled_training_set(baselist,L=5,F=1,limited_load=-1):
    if limited_load>0: baselist = baselist[:limited_load]
    X = []
    Y = []
    for P in baselist:
        x,y = split_sentence(basephrase=P,L=L,F=F)
        for k in range(len(x)): 
            X.append(x[k])
            Y.append(y[k])   
    return X,Y

In [264]:
def encode_words_by_dict(L,encode_dict):
    Le = []
    for e in L:
        ll = [encode_dict.ix[k,'Code'] for k in e]
        Le.append(ll)
    return Le

In [370]:
def clean_dataset(X,Y,MAXLEN=15):
    indices = []
    for k in range(len(Y)):
        flag = False
        for yy in Y[k]: 
            if len(yy)>MAXLEN: flag = True
        if not flag: indices.append(k)
    X = [X[k] for k in indices]
    Y = [Y[k] for k in indices]
    return X,Y

In [181]:
bf = pd.read_csv('..\\..\\TF_data\\twitter_data\\train_tweets.csv',index_col='id')

In [182]:
df = bf[['tweet']].copy()

df['tweet'] = df['tweet'].apply(lambda x: re.sub(r'[^a-zA-Z0-9_@ ]','',x))
df['tweet'] = df['tweet'].apply(lambda x: re.sub(r'\s+',' ',x.lower().strip()))

In [183]:
df['tweet'] = df['tweet'].apply(lambda x: re.sub(r'\@user @user','@user',x))
df['tweet'] = df['tweet'].apply(lambda x: re.sub(r'\@user @user @user','@user',x))
df['tweet'] = df['tweet'].apply(lambda x: re.sub(r'\@user @user @user @user','@user',x))
df['tweet'] = df['tweet'].apply(lambda x: re.sub(r'\@user @user @user @user @user','@user',x))
df['tweet'] = df['tweet'].apply(lambda x: re.sub(r'\@user @user @user @user @user @user','@user',x))
df['tweet'] = df['tweet'].apply(lambda x: re.sub(r'\@user @user @user @user @user @user @user','@user',x))
df['tweet'] = df['tweet'].apply(lambda x: re.sub(r'\@user @user @user @user @user @user @user @user','@user',x))
df['tweet'] = df['tweet'].apply(lambda x: re.sub(r'\@user @user @user @user @user @user @user @user @user','@user',x))

df['tweet'] = df['tweet'].apply(lambda x: re.sub(r'\@user@user','@user',x))
df['tweet'] = df['tweet'].apply(lambda x: re.sub(r'\@user@user@user','@user',x))
df['tweet'] = df['tweet'].apply(lambda x: re.sub(r'\@user@user@user@user','@user',x))
df['tweet'] = df['tweet'].apply(lambda x: re.sub(r'\@user@user@user@user@user','@user',x))
df['tweet'] = df['tweet'].apply(lambda x: re.sub(r'\@user@user@user@user@user@user','@user',x))
df['tweet'] = df['tweet'].apply(lambda x: re.sub(r'\@user@user@user@user@user@user@user','@user',x))
df['tweet'] = df['tweet'].apply(lambda x: re.sub(r'\@user@user@user@user@user@user@user@user','@user',x))
df['tweet'] = df['tweet'].apply(lambda x: re.sub(r'\@user@user@user@user@user@user@user@user@user','@user',x))

In [184]:
df['tweet'] = df['tweet'].apply(lambda x: x+' <EOL>')

In [320]:
wordlist = list(set([kk for k in df['tweet'].tolist() for kk in k.split(' ')]))
encode_dict = pd.DataFrame(wordlist,columns=['Word']).reset_index().rename(columns={'index':'Code'}).set_index('Word')

In [321]:
encode_dict['Word'] = encode_dict.index
encode_dict.index = encode_dict.index.rename('idx')
encode_dict['Wordlength'] = encode_dict['Word'].apply(lambda x: len(x))

In [387]:
X,Y = build_labeled_training_set(baselist=df['tweet'].tolist())

In [388]:
X,Y = clean_dataset(X,Y)

In [389]:
Y0 = encode_words_by_dict(Y,encode_dict)

In [390]:
ohe = OneHotEncoder()
Y1 = ohe.fit_transform(Y0)

In [396]:
Y0

[[9632],
 [6229],
 [2961],
 [4084],
 [15570],
 [4249],
 [38878],
 [26881],
 [4863],
 [37036],
 [26881],
 [160],
 [22523],
 [130],
 [29119],
 [38742],
 [34397],
 [29245],
 [26437],
 [38433],
 [30726],
 [14206],
 [1156],
 [1308],
 [31502],
 [19765],
 [30508],
 [130],
 [7011],
 [14458],
 [11646],
 [5118],
 [8563],
 [1308],
 [5507],
 [130],
 [36609],
 [27363],
 [29706],
 [26437],
 [19834],
 [26368],
 [6229],
 [18],
 [18593],
 [17435],
 [26437],
 [18646],
 [24257],
 [37658],
 [130],
 [16090],
 [130],
 [5118],
 [41097],
 [8303],
 [20178],
 [38742],
 [22659],
 [1539],
 [27078],
 [1691],
 [20178],
 [7794],
 [16606],
 [8709],
 [8017],
 [130],
 [17700],
 [5067],
 [10355],
 [35284],
 [130],
 [4084],
 [25899],
 [130],
 [32977],
 [1051],
 [36001],
 [3377],
 [10229],
 [11048],
 [1308],
 [17507],
 [36295],
 [29195],
 [12967],
 [31128],
 [130],
 [24672],
 [12224],
 [28596],
 [15570],
 [1020],
 [38927],
 [15673],
 [130],
 [25755],
 [31585],
 [9550],
 [5573],
 [130],
 [1691],
 [39656],
 [5118],
 [20755]